In [1]:
import os
import re
import sys
import json
import random
import itertools
from copy import copy
import numpy as np
import pandas as pd
from nltk import ngrams
from pandas.io.json import json_normalize
import keras
from keras.layers import LSTM, Dense, Bidirectional
from keras.models import Sequential, load_model
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Utils

In [2]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, sentences, next_words, maxlen, word_index, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.next_words = next_words
        self.sentences = sentences
        self.shuffle = shuffle
        self.maxlen = maxlen
        self.word_index = word_index
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.sentences) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        sentences_temp = [self.sentences[k] for k in indexes]
        next_words_temp = [self.next_words[l] for l in indexes]

        # Generate data
        X, y = self.__data_generation(sentences_temp, next_words_temp)

        return X, y
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.sentences))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, sentences, next_words):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((len(sentences), self.maxlen, len(word_index))) # (sentences)
        y = np.zeros((len(sentences), len(word_index)), dtype=np.bool)

        # Generate data
        for i, sentence in enumerate(sentences):
            for t, word in enumerate(sentence):
                X[i, t, word_index[word]] = 1    # one hot encoding
                y[i, word_index[next_words[i]]] = 1

        return X, y

def load_json(json_path, artists=[]):
    if (os.path.isfile(json_path)):
        print("json")
        with open(json_path) as f:
            song_data = json.load(f)
            return song_data['songs']
        
    elif (os.path.isdir(json_path)):
        data = []
        json_files = []
        if (len(artists) > 0):
            for artist in artists:
                json_files = json_files + [json_file for json_file in os.listdir(json_path) if ((json_file.endswith('.json')) & (artist in json_file))]
        else:
            json_files = [json_file for json_file in os.listdir(json_path) if json_file.endswith('.json')]

        for json_file in json_files:
            path_to_json = os.path.join(json_path, json_file)
            with open(path_to_json) as f:
                song_data = json.load(f)
                data = data + song_data['songs']
        
        return data
    
    
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

def normalize_lyric(text, lower=True):
    if lower:
        text = text.lower()
    text = re.sub('\[.+\](\\n)|\[.+\](\(.*\))', '', text)
    return text 

## Load Data

In [3]:
json_path = '../data/deutsch'
artists = ['Bushido']

data = load_json(json_path, artists)
df = json_normalize(data)
lyrics = df.lyrics.map(lambda lyric: normalize_lyric(lyric))

## Preprocess Data

In [4]:
lyrics_in_words = []
for lyric in lyrics:
    lyric = lyric.replace('\n', ' \n ').lower()
    words = lyric.split(' ')
    lyrics_in_words.append(words)
    
print('Corpus length in words:', len(list(itertools.chain(*lyrics_in_words))))

Corpus length in words: 58768


In [5]:
words = set(list(itertools.chain(*lyrics_in_words)))
print('Unique words:', len(words))
word_index = dict((c, i) for i, c in enumerate(words))
index_word = dict((i, c) for i, c in enumerate(words))

Unique words: 9013


In [6]:
maxlen = 7
step = 1

sentences = []
next_words = []
for lyric in lyrics_in_words:
    for i in range(0, len(lyric) - maxlen, step): # iterates by step size
        sentences.append(lyric[i: i + maxlen]) # get maxlen amount of characters
        next_words.append(lyric[i + maxlen])

## Create Datagenerators

In [7]:
sentences_train, sentences_test, next_words_train, next_words_test = train_test_split(sentences, next_words)

In [9]:
EPOCHS = 1
BATCH_SIZE = 64
DIR = '../outputs/wordbased/LSTM_Simple_WordBased_{}_E{}_BS{}_ML{}_SS{}'.format(artists[0], EPOCHS, BATCH_SIZE, maxlen, step)

if not os.path.exists(DIR):
    os.makedirs(DIR)

In [8]:
training_generator = DataGenerator(sentences_train, next_words_train, maxlen, word_index, batch_size=BATCH_SIZE)
test_generator = DataGenerator(sentences_test, next_words_test, maxlen, word_index, batch_size=BATCH_SIZE)

NameError: name 'BATCH_SIZE' is not defined

## Callbacks

In [15]:
tensorboard = TensorBoard(log_dir=os.path.join(DIR, 'logs'), write_images=True, write_grads=True)
modelCheckpoint_best = ModelCheckpoint(filepath=os.path.join(DIR, "model_best.h5"), save_best_only=True)
modelCheckpoint = ModelCheckpoint(filepath=os.path.join(DIR, "model.h5"), save_best_only=False)
#earlyStopping = EarlyStopping(patience=8)

## Build Model

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               4680704   
_________________________________________________________________
dense_2 (Dense)              (None, 9013)              1162677   
Total params: 5,843,381
Trainable params: 5,843,381
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [14]:
model.fit_generator(training_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[tensorboard, modelCheckpoint, modelCheckpoint_best])

Epoch 1/1
680/680 [==============================] - 148s 218ms/step - loss: 6.4584 - val_loss: 6.0204


In [13]:
#AUTO TRAIN
BATCH_SIZE = 64
RANGE = 10
GEN_WORD_LEN = 453
training_generator = DataGenerator(sentences_train, next_words_train, maxlen, word_index, batch_size=BATCH_SIZE)
test_generator = DataGenerator(sentences_test, next_words_test, maxlen, word_index, batch_size=BATCH_SIZE)
for it in range(1, RANGE):
    EPOCHS = 25
    DIR = '../outputs/wordbased/LSTM_Simple_WordBased_{}_E{}_BS{}_ML{}_SS{}'.format(artists[0], EPOCHS*it, BATCH_SIZE, maxlen, step)
    if not os.path.exists(DIR):
        os.makedirs(DIR)
        
    tensorboard = TensorBoard(log_dir=os.path.join(DIR, 'logs'), write_images=True, write_grads=True)
    modelCheckpoint_best = ModelCheckpoint(filepath=os.path.join(DIR, "model_best.h5"), save_best_only=True)
    modelCheckpoint = ModelCheckpoint(filepath=os.path.join(DIR, "model.h5"), save_best_only=False)

    model.fit_generator(training_generator,
        epochs=EPOCHS,
        validation_data=test_generator,
        callbacks=[tensorboard, modelCheckpoint, modelCheckpoint_best])
    
    ##########
    temperatures = [0.2, 0.4, 0.5, 0.6, 0.8, 1.]
    for temperature in temperatures:
        generated_text_temp = ["wenn", "der", "benz", "anspringt", "und", "die", "reifen"]
        generated_text = copy(generated_text_temp)
        print(" ".join(generated_text) + '_')
        #print('\n___________________\n')
        for i in range(GEN_WORD_LEN):
            sampled = np.zeros((1, maxlen, len(words)))

            for t, word in enumerate(generated_text_temp):
                sampled[0, t, word_index[word]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = index_word[next_index]
            generated_text_temp.append(next_word)
            generated_text.append(next_word)
            generated_text_temp = generated_text_temp[1:]
            #sys.stdout.write(" " + next_word)
            
        with open(os.path.join(DIR, '{}_temp{}_text.txt'.format(artists[0], temperature)), 'w+') as text_file:
            text_file.write(' '.join(generated_text))

Epoch 1/25
680/680 [==============================] - 93s 137ms/step - loss: 6.4056 - val_loss: 6.0075
Epoch 2/25
680/680 [==============================] - 110s 161ms/step - loss: 5.7705 - val_loss: 6.3474
Epoch 3/25
680/680 [==============================] - 110s 162ms/step - loss: 5.8172 - val_loss: 6.4465
Epoch 4/25
680/680 [==============================] - 107s 158ms/step - loss: 5.4178 - val_loss: 6.3681
Epoch 5/25
680/680 [==============================] - 110s 162ms/step - loss: 4.9444 - val_loss: 6.4031
Epoch 6/25
680/680 [==============================] - 109s 160ms/step - loss: 4.5167 - val_loss: 6.3972
Epoch 7/25
680/680 [==============================] - 114s 167ms/step - loss: 4.2346 - val_loss: 6.4783
Epoch 8/25
680/680 [==============================] - 94s 138ms/step - loss: 4.1270 - val_loss: 6.4382
Epoch 9/25
680/680 [==============================] - 63s 92ms/step - loss: 4.0789 - val_loss: 6.5317
Epoch 10/25
680/680 [==============================] - 103s 152ms/st

Epoch 25/25
680/680 [==============================] - 51s 75ms/step - loss: 3.3374 - val_loss: 6.6883
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
Epoch 1/25
680/680 [==============================] - 35s 52ms/step - loss: 3.3359 - val_loss: 6.6697
Epoch 2/25
680/680 [==============================] - 49s 72ms/step - loss: 3.3337 - val_loss: 6.6148
Epoch 3/25
680/680 [==============================] - 49s 72ms/step - loss: 3.3333 - val_loss: 6.6808
Epoch 4/25
680/680 [==============================] - 51s 75ms/step - loss: 3.3307 - val_loss: 6.6324
Epoch 5/25
680/680 [==============================] - 51s 75ms/step - loss: 3.3280 - val_loss: 6.5865
Epoch 6/25
680/680 [==============================] - 51s 75ms/step - loss: 3.3287 - val_loss: 6.6537
Epoch 7/25
680/680 [=========================

Epoch 23/25
680/680 [==============================] - 40s 59ms/step - loss: 3.2869 - val_loss: 6.5771
Epoch 24/25
680/680 [==============================] - 39s 58ms/step - loss: 3.2854 - val_loss: 6.6499
Epoch 25/25
680/680 [==============================] - 38s 56ms/step - loss: 3.2887 - val_loss: 6.6200
wenn der benz anspringt und die reifen_


/home/gruppe3B/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in log


wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
Epoch 1/25
680/680 [==============================] - 26s 38ms/step - loss: 3.2898 - val_loss: 6.6085
Epoch 2/25
680/680 [==============================] - 40s 58ms/step - loss: 3.2874 - val_loss: 6.5863
Epoch 3/25
680/680 [==============================] - 40s 59ms/step - loss: 3.2843 - val_loss: 6.5918
Epoch 4/25
680/680 [==============================] - 39s 58ms/step - loss: 3.2874 - val_loss: 6.6076
Epoch 5/25
680/680 [==============================] - 39s 57ms/step - loss: 3.2976 - val_loss: 6.6434
Epoch 6/25
680/680 [==============================] - 40s 58ms/step - loss: 3.2687 - val_loss: 6.5953
Epoch 7/25
680/680 [==============================] - 39s 58ms/step - loss: 3.2786 - val_loss: 6.6186
Epoch 8/25
680/680 [==============================] - 39s 57ms/step - loss: 3.2747 - v

680/680 [==============================] - 37s 54ms/step - loss: 3.2811 - val_loss: 6.6487
Epoch 25/25
680/680 [==============================] - 36s 54ms/step - loss: 3.2905 - val_loss: 6.7051
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_
wenn der benz anspringt und die reifen_


In [17]:
range(2)

range(0, 2)

## Generate Text

In [32]:
temperature = 0.5
random.seed(3004)

#lyrics_index = random.randint(0, len(lyrics))
#chosen_lyric = lyrics_in_words[lyrics_index]
#start_index = random.randint(0, len(chosen_lyric) - maxlen - 1)
#generated_text_temp = chosen_lyric[start_index: start_index + maxlen]
generated_text = ["wenn", "der", "benz", "anspringt", "und", "die", "reifen"]
generated_text_temp = copy(generated_text_temp)
print(" ".join(generated_text) + '_')
#print('\n___________________\n')
for i in range(100):
    sampled = np.zeros((1, maxlen, len(words)))
            
    for t, word in enumerate(generated_text_temp):
        sampled[0, t, word_index[word]] = 1.
                      
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_word = index_word[next_index]
    generated_text_temp.append(next_word)
    generated_text.append(next_word)
    generated_text_temp = generated_text_temp[1:]
    sys.stdout.write(" " + next_word)

wenn der benz anspringt und die reifen_
 tag 
 ich bin ein - und du bist 
 doch es ist noch immer wie ein mann 
  
  
 kannst du die es sehen, kannst du gott haben mein mit hat 
 in bleib mehr wenn der meine nicht ist 
 mein rap ist geld und mein hund 
 die es der nicht mehr einfach aus 
 mein ohne ist mehr mit als weil es zum nicht mehr wie 
 du für mir scheiße nie mehr von der - ich kannst du nicht mit mir auf sein 
 du niemals mich einfach an der vater 


In [29]:
generated_text_temp

['auf', 'dem', '\n', 'ihr', '', '\n', 'black,']

In [30]:
generated_text

['wenn',
 'der',
 'benz',
 'anspringt',
 'und',
 'die',
 'reifen',
 'immer',
 'als',
 'ein',
 'der',
 'in',
 'deinen',
 'kein',
 'im',
 '\n',
 'bist',
 'weil',
 'ein',
 'mir',
 'wenn',
 'die',
 'du',
 'bist',
 'ich',
 'nicht',
 'dich',
 'a',
 'als',
 '\n',
 'am',
 'den',
 'mir',
 'am',
 'was',
 'ihr',
 'dass',
 'ihr',
 'einfach',
 'nicht',
 'mal',
 'und',
 'die',
 'dir',
 'nicht',
 'mehr',
 '\n',
 'ihr',
 'habt',
 'alle',
 "'ner",
 'gegen',
 'auf',
 '\n',
 'mann,',
 'gibt',
 'es',
 'keine',
 'wieder',
 'war',
 '\n',
 'ich',
 'bin',
 'immer',
 'noch',
 'der',
 'nicht',
 'in',
 'meinen',
 'cool',
 '\n',
 'so',
 'dein',
 'ihr',
 'leben',
 'war',
 '\n',
 'ich',
 'hab',
 "'ne",
 'ein',
 'mit',
 'der',
 'nie',
 'mehr',
 'nach',
 'der',
 'sein',
 '\n',
 'du',
 'hast',
 'ein',
 'es',
 'gibt',
 'seine',
 "'ne",
 '\n',
 'wir',
 'sind',
 'ein',
 'auf',
 'dem',
 '\n',
 'ihr',
 '',
 '\n',
 'black,']